In [27]:
import numpy as np

In [66]:
# Generate a noisy sample vector
def generate_sample_vector(sampling_matrix, s, seed=0):
    np.random.seed(seed)
    indices = np.random.choice(sampling_matrix.shape[1], size=s, replace=False)
    selected_vectors = sampling_matrix[:, indices]
    coefficients = np.random.normal(size=(s, 1))  # random coefficients for each selected vector
    y = selected_vectors @ coefficients
    return y, indices, coefficients

def generate_noisy_sample_vector(y, noise_level, seed=0):
    np.random.seed(seed)
    norm_y = np.linalg.norm(y)
    noise = np.random.normal(size=y.shape, scale=norm_y * noise_level)
    y_perturbed = y + noise
    return y_perturbed

In [88]:
# Generate input params for CoSaMP
np.random.seed(0)
N = 100000
d = 300
s = 3
sampling_matrix = np.random.normal(size=(N,d))
u, indices, coefficients = generate_sample_vector(sampling_matrix, s)
u_pertubed = generate_noisy_sample_vector(u, 0.1)

In [89]:
indices

array([208, 188,  12])

In [90]:
coefficients

array([[ 1.12663592],
       [-1.07993151],
       [-1.14746865]])

In [115]:
def CoSaMP(phi, u, s, n):

    # Initialize params
    prune_indice = np.array([], dtype=int)
    v = u.copy()
    k = 0

    while k < n:
        k += 1

        print(f'Iteration: {k}')

        # Form signal proxy
        y = phi.T @ v

        # Identify large components
        omega = np.argsort(np.abs(y), axis=0)[-2*s:, :][:, 0]
        
        # Merge supports
        T = np.union1d(omega, prune_indice)

        # Signal estimation by least-squares
        b = np.zeros_like(y)
        phi_T = phi[:,T]
        b[T] = np.linalg.inv(phi_T.T @ phi_T) @ phi_T.T @ u

        # Prune to obtain next approximation
        a_k = np.zeros_like(y)
        prune_indice = np.argsort(np.abs(b), axis=0)[-s:, :][:, 0]
        a_k[prune_indice] = b[prune_indice]

        # Update current samples
        v = u - phi @ a_k
    
    return prune_indice, a_k[prune_indice]

In [116]:
output_indice, output_coefficient = CoSaMP(sampling_matrix, u_pertubed, 3, 10)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10


In [119]:
output_indice

array([188,  12, 208], dtype=int64)

In [118]:
output_coefficient

array([[-0.95830363],
       [-1.11116992],
       [ 1.27176295]])